# Name

James Troy

# Set Up


### Database
The database schema is in a file called `setup.sql`

Create the database, you can use your own database, or the docker container included. To run the docker container
do `docker-compose up`, then `mysql -h 172.33.0.100 -u root -p  < "create_db.sql"`.

If using an alternative database use the following command `mysql -h <host>> -u <user>> -p  < "create_db.sql"`

    mysql -h 172.33.0.100 -u root -p  < "create_db.sql"

### Configuration

The [config.ini](config.ini) file contains database configuration, change it if you want to use some other credentials.


### How it works

#### Scraping
There is a table module that has two classes, `PrimeMinister` & `PrimeMinisterTerms`.
The `PrimeMinisterTerms` scrapes the main wikipedia prime ministers table and creates a matrix that reflects
what the contents of the table is.

The `PrimeMinister` retrieves data from the prime ministers personal page on wikipedia, here it extracts the
date of birth and death if any

#### Configuration

The database has 5 tables, `party`, `monarch`, `prime_minister`, `prime_minister_term` and `term_monarch`.

* **The default host that  points to the docker container  is `172.33.0.100`**
* **The default username is `nutty_professor`**
* **The default password is `scientist`**
* **The database name is `assignment2`**

***
Please change the host for the database in [config.ini](config.ini)
***

### Getting the data

Invoke the function `main()` in the core module.

Alternatively run

    python core.py

This scrapes the data and populates the database with it.



## Helper function `calculate_total_time` to calculate  years, days & months.
### Presumptions
* presume 365 days in a year
* 30 days in a month

***

Some basic setup

In [ ]:
import configparser
import os
from core import main, calculate_total_time

from dateutil import relativedelta
from database import MariaDBConnector

config = configparser.ConfigParser()
ini_path = os.path.join(os.getcwd(), "config.ini")
config.read(ini_path)
db_config = config["DATABASE"]

Populate the database


In [ ]:
main()


a) Which political party has produced the most individual prime ministers? [2]


In [ ]:
with MariaDBConnector(**db_config) as cursor:
    query = """
        select party_name, max(total) from
            (select party_name, count(*) as total
                from prime_minister
                join party p on prime_minister.party_id = p.id
                group by  party_name
                order by total desc) as party_count;
        """

    cursor.execute(query)
    db_response = cursor.fetchall()

    print(
        f"The party that produced the most Prime Ministers is the {db_response[-1][0]} party"
    )

b) Provide a list of prime ministers who served under each monarch. [3]

In [ ]:
with MariaDBConnector(**db_config) as cursor:
    query = """
        select  m.title, pm.name from prime_minister_term
            join term_monarch tm on prime_minister_term.id = tm.term_id
            join monarch m on tm.monarch_id = m.id
            join prime_minister pm on pm.id = prime_minister_term.prime_minister_id;
    """

    cursor.execute(query)
    db_response = cursor.fetchall()
    monarch_pms = {}

for row in db_response:
    monarch = row[0]
    pms = monarch_pms.get(monarch, [])
    pms.append(row[1])
    monarch_pms[monarch] = pms

for k, v in monarch_pms.items():
    print(f"{k} -> {list(set(v))}")

(c) Which political party held the office of prime minister for the longest amount of
overall time, and for how long (in years, months, and days)? [3]


In [ ]:
query = """
    SELECT party_name, start_date, IFNULL(end_date, CURRENT_DATE) end_date,
           datediff(end_date, start_date) as term_days
    from prime_minister_term
      join prime_minister pm on prime_minister_term.prime_minister_id = pm.id
      join party p on pm.party_id = p.id
      where party_name = (
        select party_name from(
          select party_name, max(sum_terms) from (
            SELECT party_name, SUM(term_days) as sum_terms from (
              SELECT party_name, start_date, IFNULL(end_date, CURRENT_DATE) end_date,
              datediff(end_date, start_date) as term_days  

              from prime_minister_term
                join prime_minister pm on prime_minister_term.prime_minister_id = pm.id
                join party p on pm.party_id = p.id
        ) as t1
        group by party_name)
    as t2) 
as t3);
"""

with MariaDBConnector(**db_config) as cursor:
    cursor.execute(query)
    db_response = cursor.fetchall()

deltas = {"years": 0, "months": 0, "days": 0, "term_in_days": 0}

for row in db_response:
    party = row[0]
    start = row[1]
    end = row[2]
    if row[3]:
        term_in_days = row[3]
    else:
        term_in_days = (date.today() - start).days
    _delta = relativedelta.relativedelta(end, start)
    deltas["years"] += _delta.years
    deltas["months"] += _delta.months
    deltas["days"] += _delta.days
    deltas["term_in_days"] += term_in_days


years, months, days, total_days_check = calculate_total_time(
    deltas["years"], deltas["months"], deltas["days"], deltas["term_in_days"]
)

print(
    f"The {party} party held the office of prime minister "
    f"for the longest amount of overall time, "
    f"{years} years, {months} months and {days} days ..."
)

(d) Which individual politician held the office of prime minister for the longest amount
of uninterrupted time, and for how long (in years, months, and days)? [2]

In [ ]:
query = """
SELECT pm.name, start_date, IFNULL(end_date, CURRENT_DATE) end_date,
   max(datediff(IFNULL(end_date, CURRENT_DATE), start_date) )as term_days
from prime_minister_term
    join prime_minister pm on pm.id = prime_minister_term.prime_minister_id;
"""

with MariaDBConnector(**db_config) as cursor:

    db_response = cursor.query(query)[-1]
name = db_response[0]
start = db_response[1]
end = db_response[2]
_delta = relativedelta.relativedelta(end, start)
years = _delta.years
months = _delta.months
days = _delta.days
print(
    f"The individual politician that held the office of prime minister "
    f"for the longest amount of uninterrupted time is \n"
    f"{name} for {years} years, {months} months, and {days} days..."
)

(e) Which individual politician has held the office of prime ministers for the longest
amount of overall time? [1]


In [ ]:
query = """
SELECT pm.name as pm_name,
    sum(datediff(IFNULL(end_date, CURRENT_DATE), start_date)) as overall_days
from prime_minister_term
    join prime_minister pm on pm.id = prime_minister_term.prime_minister_id
group by pm.id
order by overall_days desc
LIMIT 1;"""
with MariaDBConnector(**db_config) as cursor:
    cursor.execute(query)
    db_response = cursor.fetchall()
pm = db_response[0]

name = pm[0]
total_days = pm[-1]
years, months, days, total_days_check = calculate_total_time(0, 0, 0, total_days)

print(
    f"The individual politician that held the office \n"
    f"of prime minister for the longest amount of \n"
    f"uninterrupted time is \n"
    f"{name} for {years} years, {months} months, and {days} days..."
)

(f) Which individual politician held the office of prime minister for the shortest amount
of overall time, and for how long (in years, months, and days)? [1]


In [ ]:
query = """
SELECT pm.name                                                   as pm_name,
   start_date,
   IFNULL(end_date, CURRENT_DATE)                               end_date,
   sum(datediff(IFNULL(end_date, CURRENT_DATE), start_date)) as overall_days
from prime_minister_term
         join prime_minister pm on pm.id = prime_minister_term.prime_minister_id
group by pm.id
order by overall_days asc
LIMIT 1"""
with MariaDBConnector(**db_config) as cursor:
    cursor.execute(query)
    db_response = cursor.fetchall()


pm = db_response[0]
name = pm[0]
delta = relativedelta.relativedelta(pm[2], pm[1])
years = delta.years
months = delta.months
days = delta.days


print(
    f"The individual politician that held the office \n"
    f"of prime minister for the shortest amount of \n"
    f"uninterrupted time is {name} "
    f"for {years} years, {months} months, and {days} days..."
)

(g) List the prime ministers who where either an earl, viscount, duke, or knight? [4]


In [ ]:
query = """
select name, alias, title
from prime_minister_term
         join prime_minister pm on prime_minister_term.prime_minister_id = pm.id
where not title  = ''
group by
pm.id
order by name;
"""
with MariaDBConnector(**db_config) as cursor:
    cursor.execute(query)
    db_response = cursor.fetchall()

for name, alias, title in db_response:
    _title = f"{name}, {alias}" if title.lower() != "knight" else f"{alias} {name}"

    print(_title)

What age was each prime minister on the day they assumed office? [4]


In [ ]:
query = """
select pm.id, pm.name, birth_date, min(start_date),
       CAST((DATE_FORMAT(FROM_DAYS( DATEDIFF(start_date,birth_date)), '%Y')+0) AS UNSIGNED ) AS age
from prime_minister_term
         join prime_minister pm on pm.id = prime_minister_term.prime_minister_id
group by pm.id
order by age desc;
"""
with MariaDBConnector(**db_config) as cursor:
    cursor.execute(query)
    db_response = cursor.fetchall()


for pm in db_response:

    print(f"{pm[1]} was {pm[-1]} years old assuming office for the first time")


On the last day of their term of office, which politician was the oldest? [1]

In [ ]:
query = """
select pm.id,
       pm.name,
       birth_date,
       max(IFNULL(end_date, CURRENT_DATE))                     end_date,
       CAST((DATE_FORMAT(FROM_DAYS(DATEDIFF((IFNULL(end_date, CURRENT_DATE)), birth_date)), '%Y') +
             0) AS UNSIGNED)                                AS age,

       datediff(IFNULL(end_date, CURRENT_DATE), birth_date) as overall_days
from prime_minister_term
         join prime_minister pm on pm.id = prime_minister_term.prime_minister_id
group by pm.id
order by overall_days desc
limit 1;
"""
with MariaDBConnector(**db_config) as cursor:
    cursor.execute(query)
    db_response = cursor.fetchall()

pm = db_response[0]

print(f"{pm[1]} was {pm[4]} years old when leaving office, thats {pm[5]} days!")